In [1]:
import cv2
import json
import numpy as np
import chess
import chess.svg
from cairosvg import svg2png
import random 
import chess.engine
import time
from Arm_Lib import Arm_Device
import asyncio

engine = chess.engine.SimpleEngine.popen_uci('Stockfish/src/./stockfish' )

with open('coord2.json', 'r') as fp:
    sq_points = json.load(fp)

# Get DOFBOT object
Arm = Arm_Device()
time.sleep(.1)

In [2]:
def find_square(x: float, y: float, sq_points: dict):
    for square in sq_points:
        points = np.array(sq_points[square], np.int32)
        if cv2.pointPolygonTest(points, (x, y), False) > 0:
            return square
    return None

In [3]:
def draw_outlines(sq_points: dict, frame, show_text = False) -> None:
    for square in sq_points:
        points = np.array(sq_points[square], dtype=np.int32)
        cv2.polylines(frame,[points], True, (255, 255, 255), thickness=1)
        x, y, w, h = cv2.boundingRect(points)
        if show_text:
            cv2.putText(frame, square, (x + w //2, y + h //2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

In [4]:
def show_board(board: chess.Board, size=600, move = None) -> None:
    if move is not None:
        sq1, sq2 = chess.parse_square(move[:2]), chess.parse_square(move[2:4])
        svgwrap = chess.svg.board(board, size=size, fill=dict.fromkeys([sq1, sq2], '#ced264'))
    else:
        svgwrap = chess.svg.board(board, size=size)
    svg2png(bytestring=svgwrap, write_to='output.png')
    cv2.imshow('Game', cv2.imread('output.png'))

In [7]:
async def pick_piece(array, speed=1700):
    try:
        Arm.Arm_serial_servo_write6_array(array, speed)
        og = array[5]
        await asyncio.sleep(5)  # Pause for 5 seconds asynchronously
        array[5] = 179
        Arm.Arm_serial_servo_write6_array(array, speed)
        await asyncio.sleep(1)  # Pause for 1 second asynchronously
        array[5] = og
        print(array)
        print("Press 'u' to continue...")
    except ValueError as e:
        print(f"Invalid move at index ('{array}'): {e}")

   
            

In [8]:
async def place_piece(array, speed=1700):
    try:
        Arm.Arm_serial_servo_write6_array(array, speed)
        await asyncio.sleep(3)  # Pause for 3 seconds asynchronously
        array[5] = 159
        Arm.Arm_serial_servo_write6_array(array, speed)
        print(array)
    except ValueError as e:
        print(f"Invalid move at index ('{array}'): {e}")

            

In [9]:
async def process_chess_moves():
    board = chess.Board()  # Initialize the chess board
    show_board(board) 
    await asyncio.sleep(25)
    moves = ["e2e4", "e7e5", "g1f3", "b8c6", "f1c4", "f8c5"]

    for move in moves:
        try:
            board.push_uci(move)  # Push the move onto the chess board
            show_board(board, size=400, move=move)  # Display the updated board
            await asyncio.sleep(15)
        except ValueError as e:
            print(f"Invalid move '{move}': {e}")
        await asyncio.sleep(0.1)  # Prevent blocking other tasks


In [10]:
async def robotic_arm_operations():
    array = [
        [93, 160, 0, 0, 90, 180],
        [91, 65, 36, 5, 83, 159],
        [90, 54, 47, 13, 89, 178],
        [71, 77, 20, 9, 68, 162],
        [82, 71, 16, 25, 80, 179],
        [81, 84, 12, 10, 78, 162],
        [102, 46, 57, 5, 90, 179]
    ]
    

    await rest_arm(array[0])
    await asyncio.sleep(5)

    await pick_piece(array[1])
    await asyncio.sleep(5)

    await rest_arm(array[0])
    await place_piece(array[2])
    await asyncio.sleep(10)

    await rest_arm(array[0])
    await asyncio.sleep(5)

    await pick_piece(array[3])
    await asyncio.sleep(5)

    await rest_arm(array[0])
    await place_piece(array[4])
    await asyncio.sleep(10)

    await rest_arm(array[0])
    await asyncio.sleep(5)

    await pick_piece(array[5])
    await asyncio.sleep(5)

    await rest_arm(array[0])
    await place_piece(array[6])
    await asyncio.sleep(5)

    await rest_arm(array[0])

In [11]:
async def capture_and_display_camera(cap):
    """Asynchronously capture and display camera frames."""
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame. Exiting...")
            break
        draw_outlines(sq_points, frame)
        # Display the frame
        cv2.imshow('Frame', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Add a small sleep to prevent CPU overuse
        await asyncio.sleep(0.01)

In [12]:
async def main():
     cap = cv2.VideoCapture(0)

     if not cap.isOpened():
        print("Failed to open camera.")
        return# Create tasks for robotic arm operations and chess moves
     camera_task = asyncio.create_task(capture_and_display_camera(cap))
     arm_task = asyncio.create_task(robotic_arm_operations())
     chess_task = asyncio.create_task(process_chess_moves())
    
        # Wait for both tasks to complete
    
     await asyncio.gather(camera_task, arm_task, chess_task)

    # Release resources
     cap.release()
     cv2.destroyAllWindows()
    
    # Run the main function
import nest_asyncio
nest_asyncio.apply()
    
await main()


[91, 65, 36, 5, 83, 159]
Press 'u' to continue...
[90, 54, 47, 13, 89, 159]
[71, 77, 20, 9, 68, 162]
Press 'u' to continue...
[82, 71, 16, 25, 80, 159]
[81, 84, 12, 10, 78, 162]
Press 'u' to continue...
[102, 46, 57, 5, 90, 159]


In [13]:

# import time


# # Create a chess board
# board = chess.Board()

# # Array of moves in UCI format
# moves = ["e2e4", "e7e5", "g1f3", "b8c6"]
# import time
# import chess
# import chess.svg
# import cv2
# from cairosvg import svg2png

# # Assuming the `show_board` function is defined as before

# # Initialize chess board
# board = chess.Board()

# # Predefined array of moves in UCI format
# moves = ["e2e4", "e7e5", "g1f3", "b8c6", "f1c4", "f8c5"]
# show_board(board)
# # Iterate through the moves
# for move in moves:
#     try:
  
#         # Push the move onto the chess board"
#         board.push_uci(move)
        
#         # Display the updated chess board
#         show_board(board, size=400, move=move)
        
#         # Wait for 10 seconds before processing the next move
#         print(" press u") 
#         while True:
#             if cv2.waitKey(2) & 0xFF == ord('u'):
#                 break
    
#     except ValueError as e:
#         print(f"Invalid move '{move}': {e}")

